## Simulating Eye Sight

In [36]:
import cv2
import numpy as np
import os
import glob

def simulate_vd(image, spherical_power, cylindrical_power, axis_orientation):
    # Apply spherical correction
    simulated_image = image.copy()
    if spherical_power != 0:
        if spherical_power < 0:
            # Simulate myopia (nearsightedness)
            blur_kernel_size = int(abs(spherical_power) * 4 + 1)
            blur_kernel_size = blur_kernel_size if blur_kernel_size % 2 == 1 else blur_kernel_size + 1
            simulated_image = cv2.GaussianBlur(simulated_image, (blur_kernel_size, blur_kernel_size), 0)
        else:
            # Simulate hypermetropia (farsightedness)
            focal_length = 1 / spherical_power
            simulated_image = cv2.resize(simulated_image, None, fx=focal_length, fy=focal_length, interpolation=cv2.INTER_LINEAR)
            simulated_image = cv2.resize(simulated_image, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_LINEAR)
    
    # Apply cylindrical correction
    if cylindrical_power != 0:
        kernel_size = int(abs(cylindrical_power) * 4 + 1)
        kernel_size = kernel_size if kernel_size % 2 == 1 else kernel_size + 1
        sigma = kernel_size / 4
        lambd = kernel_size / 2
        gamma = 0.5
        kernel = cv2.getGaborKernel((kernel_size, kernel_size), sigma, np.deg2rad(axis_orientation), lambd, gamma, 0, ktype=cv2.CV_32F)
        kernel /= 1.5 * kernel.sum()
        simulated_image = cv2.filter2D(simulated_image, -1, kernel)
    
    return simulated_image

def process_images(input_folder, output_folder, spherical_power, cylindrical_power, axis_orientation, num_images):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of image files in the input folder
    image_files = glob.glob(os.path.join(input_folder, '*.png'))
    image_files += glob.glob(os.path.join(input_folder, '*.jpg'))  # Add support for jpg files
    
    # Process specified number of images
    for i, image_file in enumerate(image_files[:num_images]):
        # Load the image
        image = cv2.imread(image_file)
        
        # Simulate visual defect
        simulated_image = simulate_vd(image, spherical_power, cylindrical_power, axis_orientation)
        
        # Construct the output file path
        output_file = os.path.join(output_folder, f'Simulated_Image_{i+1}.png')
        
        # Save the simulated image
        cv2.imwrite(output_file, simulated_image)

# Get input from the user
input_folder = 'flickr30k_images'
output_folder = input("Enter the Output File Name")
spherical_power = float(input("Enter the spherical power (in diopters): "))
cylindrical_power = float(input("Enter the cylindrical power (in diopters): "))
axis_orientation = float(input("Enter the axis orientation (in degrees): "))
num_images = int(input("Enter the number of images to process: "))

# Process the images with the given metrics
process_images(input_folder, output_folder, spherical_power, cylindrical_power, axis_orientation, num_images)

Enter the Output File NameS3A3175
Enter the spherical power (in diopters): 3
Enter the cylindrical power (in diopters): 2
Enter the axis orientation (in degrees): 175
Enter the number of images to process: 10


## Correcting Eye Sight

In [5]:
import cv2
import numpy as np
import os
import glob

def apply_correction(image, spherical_power, cylindrical_power, axis_orientation):
    # Define the dimensions of the image
    height, width = image.shape[:2]

    # Create a meshgrid of coordinates
    x, y = np.meshgrid(np.arange(width), np.arange(height))

    # Convert spherical and cylindrical power to meters
    spherical_power *= -0.001  # Convert diopters to meters
    cylindrical_power *= -0.001  # Convert diopters to meters

    # Convert cylindrical power and axis orientation to radians
    cylindrical_power_rad = cylindrical_power * 2 * np.pi
    angle_rad = np.deg2rad(axis_orientation)

    # Compute the distorted coordinates
    x_distorted = x - width / 2
    y_distorted = y - height / 2

    x_corrected = x_distorted + spherical_power * x_distorted + cylindrical_power_rad * np.cos(2 * angle_rad) * x_distorted + cylindrical_power_rad * np.sin(2 * angle_rad) * y_distorted
    y_corrected = y_distorted + spherical_power * y_distorted + cylindrical_power_rad * np.sin(2 * angle_rad) * x_distorted + cylindrical_power_rad * np.cos(2 * angle_rad) * y_distorted

    x_corrected += width / 2
    y_corrected += height / 2

    # Interpolate the corrected coordinates to obtain the corrected image
    corrected_image = cv2.remap(image, x_corrected.astype(np.float32), y_corrected.astype(np.float32), cv2.INTER_LINEAR)

    return corrected_image

def save_images(image, corrected_image, output_folder, index):
    # Save original image
    original_path = os.path.join(output_folder, f'Original_Image_{index}.jpg')
    cv2.imwrite(original_path, image)

    # Save corrected image
    corrected_path = os.path.join(output_folder, f'Corrected_Image_{index}.jpg')
    cv2.imwrite(corrected_path, corrected_image)

def process_images(input_folder, output_folder, spherical_power, cylindrical_power, axis_orientation, num_images):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of image files in the input folder
    image_files = glob.glob(os.path.join(input_folder, '*.png'))
    image_files += glob.glob(os.path.join(input_folder, '*.jpg'))  # Add support for jpg files
    
    # Process specified number of images
    for i, image_file in enumerate(image_files[:num_images]):
        # Load the image
        image = cv2.imread(image_file)
        
        # Apply correction
        corrected_image = apply_correction(image.copy(), spherical_power, cylindrical_power, axis_orientation)
        
        # Save original and corrected images
        save_images(image, corrected_image, output_folder, i+1)

if __name__ == "__main__":
    # Get input from the user
    input_folder = '/Users/suhas/Documents/Education/Computer Vision/Project/Code/Datasets/Image Captioning/flickr30k_images/flickr30k_images'
    output_folder = input("Enter the output folder name")
    spherical_power = float(input("Enter spherical power (diopters): "))
    cylindrical_power = float(input("Enter cylindrical power (diopters): "))
    axis_orientation = float(input("Enter axis orientation (degrees): "))
    num_images = int(input("Enter the number of images to process: "))

    # Process the images with the given metrics
    process_images(input_folder, output_folder, spherical_power, cylindrical_power, axis_orientation, num_images)


## Simulating Color Blindness

In [ ]:

from PIL import Image
import numpy as np
import os
import glob

def simulate_color_blindness(image_array, type):
    # Apply color blindness simulation based on type
    if type == 'deuteranomaly':
        # Simulate deuteranomaly
        image_array = simulate_deuteranomaly(image_array)
    elif type == 'protanomaly':
        # Simulate protanomaly
        image_array = simulate_protanomaly(image_array)
    elif type == 'protanopia':
        # Simulate protanopia
        image_array = simulate_protanopia(image_array)
    elif type == 'deuteranopia':
        # Simulate deuteranopia
        image_array = simulate_deuteranopia(image_array)
    elif type == 'tritanomaly':
        # Simulate tritanomaly
        image_array = simulate_tritanomaly(image_array)
    elif type == 'tritanopia':
        # Simulate tritanopia
        image_array = simulate_tritanopia(image_array)
    elif type == 'monochromacy':
        # Simulate monochromacy
        image_array = simulate_monochromacy(image_array)

    return image_array

def save_images(image_path, simulated_image, output_folder, index):
    # Save original image
    original_path = os.path.join(output_folder, f'Original_Image_{index}.jpg')
    image_path.save(original_path)

    # Save simulated image
    simulated_path = os.path.join(output_folder, f'Simulated_Image_{index}.jpg')
    simulated_image.save(simulated_path)

def process_images(input_folder, output_folder, type, num_images):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get a list of image files in the input folder
    image_files = glob.glob(os.path.join(input_folder, '*.jpg'))
    
    # Process specified number of images
    for i, image_file in enumerate(image_files[:num_images]):
        # Load the image
        image = Image.open(image_file)
        
        # Convert image to numpy array
        image_array = np.array(image)
        
        # Apply color blindness simulation
        simulated_array = simulate_color_blindness(image_array.copy(), type)
        
        # Convert numpy array back to image
        simulated_image = Image.fromarray(simulated_array)
        
        # Save original and simulated images
        save_images(image, simulated_image, output_folder, i+1)


def simulate_deuteranomaly(image_array):
    # Simulate deuteranomaly (make certain shades of green look more red)
    # Adjust green component
    image_array[:, :, 1] = 0.5 * image_array[:, :, 1] + 0.5 * image_array[:, :, 0]
    return image_array

def simulate_protanomaly(image_array):
    # Simulate protanomaly (make certain shades of red look more green and less bright)
    # Adjust red component
    image_array[:, :, 0] = 0.5 * image_array[:, :, 0] + 0.5 * image_array[:, :, 1]
    # Decrease brightness
    image_array *= 0.75
    return image_array

def simulate_protanopia(image_array):
    # Simulate protanopia (unable to tell the difference between red and green at all)
    # Set red and green channels to average of blue
    average_blue = np.mean(image_array[:, :, 2])
    image_array[:, :, 0] = average_blue
    image_array[:, :, 1] = average_blue
    return image_array

def simulate_deuteranopia(image_array):
    # Simulate deuteranopia (unable to tell the difference between red and green at all)
    # Set red and green channels to average of blue
    average_blue = np.mean(image_array[:, :, 2])
    image_array[:, :, 0] = average_blue
    image_array[:, :, 1] = average_blue
    return image_array

def simulate_tritanomaly(image_array):
    # Simulate tritanomaly (make it hard to tell the difference between blue and green and between yellow and red)
    # Reduce blue component
    image_array[:, :, 2] *= 0.8
    return image_array

def simulate_tritanopia(image_array):
    # Simulate tritanopia (unable to tell the difference between blue and green, purple and red, and yellow and pink)
    # Set blue and green channels to average of red
    average_red = np.mean(image_array[:, :, 0])
    image_array[:, :, 1] = average_red
    image_array[:, :, 2] = average_red
    return image_array

def simulate_monochromacy(image_array):
    # Simulate monochromacy (complete color blindness)
    # Convert image to grayscale
    grayscale_image = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])
    # Expand grayscale to 3 channels
    monochrome_image = np.stack((grayscale_image,) * 3, axis=-1)
    return monochrome_image
 
image_path = '/Users/suhas/Documents/Education/Computer Vision/Project/Code/static/Assets/images/high res/9.jpg'
simulated_image = simulate_color_blindness(image_path, 'tritanophia')
cv2.imwrite("Simulated.jpg", simulated_image)
  


## Image Color Annotation

In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter

def load_color_dataset(dataset_path):
    color_dataset = {}
    with open(dataset_path, 'r') as file:
        for line in file:
            color_name, r, g, b = line.strip().split(',')
            color_dataset[(int(r), int(g), int(b))] = color_name
    return color_dataset

def annotate_colors(image_path, num_colors, color_dataset):
    image = cv2.imread(image_path)

    pixels = image.reshape(-1, 3)

    kmeans = KMeans(n_clusters=num_colors, random_state=42)
    kmeans.fit(pixels)

    colors = kmeans.cluster_centers_.astype(int)

    labels = kmeans.labels_

    total_pixels = image.shape[0] * image.shape[1]
    pixel_percentages = [count / total_pixels for count in Counter(labels).values()]
    sorted_colors = sorted(zip(pixel_percentages, colors), reverse=True)

    annotated_image = image.copy()

    for pixel_percentage, color in sorted_colors:
        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        mask[labels.reshape(image.shape[:2]) == sorted_colors.index((pixel_percentage, color))] = 255
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0 or cv2.contourArea(max(contours, key=cv2.contourArea)) < 100:
            continue

        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
        cv2.rectangle(annotated_image, (x, y), (x + w, y + h), color.tolist(), 2)

        closest_color_name = find_closest_color(color, color_dataset)
        cv2.putText(annotated_image, f"{closest_color_name} ({int(pixel_percentage * 100)}%)", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color.tolist(), 2)

    return annotated_image

def find_closest_color(rgb, color_dataset):
    min_dist = float('inf')
    closest_color_name = None
    for color, name in color_dataset.items():
        dist = np.linalg.norm(np.array(rgb) - np.array(color))
        if dist < min_dist:
            min_dist = dist
            closest_color_name = name
    return closest_color_name

image_path = "/Users/suhas/Documents/Education/Computer Vision/Project/Code/static/Assets/images/high res/1110.jpg"
num_colors = 5
dataset_path = "/Users/suhas/Documents/Education/Computer Vision/Project/Code/static/Assets/colors-full.csv" 

color_dataset = load_color_dataset(dataset_path)
annotated_image = annotate_colors(image_path, num_colors, color_dataset)

cv2.imwrite('Annotated Image.jpg', annotated_image)